In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point
import unicodedata
import plotly.express as px

In [2]:
assessment = gpd.read_file('./home/data/civ-assessments-v6.geojson', encoding='iso-8859-1')

In [3]:
assessment.columns

Index(['Opportunity Unique Identifier', 'Bridge Name', 'Country',
       'Level 1 Government', 'Level 2 Government', 'Level 3 Government',
       'Level 4 Government', 'latitude', 'longitude', 'Reason site not valid',
       'Site Validation Comments', 'Flag for Rejection',
       'River crossing deaths in last 3 years', 'Incident descriptions',
       'Days per year river is flooded', 'Flood duration during rainy season',
       'Notes on social information', 'Width of River During Flooding (m)',
       'Health access blocked by river', 'Education access blocked by river',
       'Market access blocked by river', 'Other access blocked by river',
       'Form: Created Date', 'Form: Created By', 'geometry'],
      dtype='object')

In [4]:
assessment

,Opportunity Unique Identifier,Bridge Name,Country,Level 1 Government,Level 2 Government,Level 3 Government,Level 4 Government,latitude,longitude,Reason site not valid,...,Flood duration during rainy season,Notes on social information,Width of River During Flooding (m),Health access blocked by river,Education access blocked by river,Market access blocked by river,Other access blocked by river,Form: Created Date,Form: Created By,geometry
0,1043157,Sangarekro,Cote d'Ivoire,San Pedro,San-Pédro,Dogbo,Bledie,4.855737,-6.969818,,...,5,Secrétaire Gsc,13,Clinique ou centre de santé - Clinic or Health...,École Primaire - Primary,Marché journalier - Market (daily);Marché hebd...,"Lieu de culte (Eglise, Mosquée, Temple, etc) -...",11/11/22,achilleyaoyao taroworks,POINT (-6.96982 4.85574)
1,1040872,Hanier,Cote d'Ivoire,San Pedro,San-Pédro,Grand bereby,Mossikro,4.718922,-6.939454,,...,35,2,16,,,,,6/30/22,achilleyaoyao taroworks,POINT (-6.93945 4.71892)
2,1042870,campement ardjoumakro,Cote d'Ivoire,San Pedro,San-Pédro,Dogbo,Bledie,4.857928,-7.000859,,...,2,Président Gsc bledie,23,Hôpital - Hospital;Pharmacie - Pharmacy,École Primaire - Primary,Marché journalier - Market (daily);Marché hebd...,"Lieu de culte (Eglise, Mosquée, Temple, etc) -...",11/7/22,achilleyaoyao taroworks,POINT (-7.00086 4.85793)
3,1042875,Sangarekro,Cote d'Ivoire,San Pedro,San-Pédro,Dogbo,Bledie,4.859334,-6.988984,,...,3,,135,Clinique ou centre de santé - Clinic or Health...,École Primaire - Primary,Marché journalier - Market (daily);Marché hebd...,"Lieu de culte (Eglise, Mosquée, Temple, etc) -...",11/7/22,achilleyaoyao taroworks,POINT (-6.98898 4.85933)
4,1042877,Bledie,Cote d'Ivoire,San Pedro,San-Pédro,Dogbo,Bledie,4.869407,-7.005066,,...,3,Président Gsc bledie,34,Clinique ou centre de santé - Clinic or Health...,École Primaire - Primary,Marché journalier - Market (daily);Marché hebd...,"Lieu de culte (Eglise, Mosquée, Temple, etc) -...",11/7/22,achilleyaoyao taroworks,POINT (-7.00507 4.86941)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,1044019,Afotobo 5,Cote d'Ivoire,Gbeke,Béoumi,Béoumi,Afotobo,7.645650,-5.477423,,...,1 jour,La communauté traverse ce site pour accéder au...,25,,,Feme - Farms,"Autre - (si sélectionné, expliquez la raison d...",12/25/22,sekredoukobénan taroworks,POINT (-5.47742 7.64565)
565,1044023,Obenikro,Cote d'Ivoire,Gbeke,Béoumi,Béoumi,Akadiafoué,7.604121,-5.484128,,...,"3 jours de crue et d?inondation, mais les élè...",Les élèves de Obenikro traversent le site pour...,15,,École Primaire - Primary,Marché hebdomadaire - Market (weekly);Feme - F...,"Lieu de culte (Eglise, Mosquée, Temple, etc) -...",12/25/22,sekredoukobénan taroworks,POINT (-5.48413 7.60412)
566,1044013,Assekro 2,Cote d'Ivoire,Gbeke,Béoumi,Béoumi,Afotobo,7.656134,-5.392766,,...,7 jours,La communauté traverse ce site pour accéder au...,125,,,Feme - Farms,,12/25/22,sekredoukobénan taroworks,POINT (-5.39277 7.65613)
567,1044184,Agba-Kouassikro 2,Cote d'Ivoire,Gbeke,Béoumi,Bodokro,Kouakou-Bakakro,7.937529,-5.474987,,...,1 jour,"Monsieur N?dri Konan Jean-François, Secrétaire...",41,,,Feme - Farms,,12/30/22,sekredoukobénan taroworks,POINT (-5.47499 7.93753)


In [5]:
# print unique values for Level 1 Government
print(len(assessment['Level 1 Government'].unique()))
print(len(assessment['Level 2 Government'].unique()))
print(len(assessment['Level 3 Government'].unique()))
print(len(assessment['Level 4 Government'].unique()))
print(len(assessment))

2
6
37
137
569


In [6]:
# create a dataframe with unique values of 'level 4 Government' as a column with the sum of times it's mentioned as a column
villages = pd.DataFrame(assessment['Level 4 Government'].value_counts())
villages = villages.reset_index()
villages.columns = ['Level 4 Government', 'Count']
villages

,Level 4 Government,Count
0,Angokoun-Kpangbassou,38
1,Andokro-Kouakou,29
2,Adjougouansou,18
3,Hannié,18
4,Bledie,17
...,...,...
132,Tanou-Broukro (Bassin de Saminikro),1
133,Ahouzankro (Bassin de Mamini),1
134,Takassou (Bassin de Mamini),1
135,Koubékro (Bassin Mamini),1


In [7]:
# for each value in villages level 4 government, find the corresponding value in the assessment level 4 government column and add the corresponding value in the "level 3 government" column as a new column in the villages dataframe
for i in range(len(villages)):
    villages.loc[i, 'Level 3 Government'] = assessment.loc[assessment['Level 4 Government'] == villages.loc[i, 'Level 4 Government'], 'Level 3 Government'].iloc[0]
    villages.loc[i, 'Level 2 Government'] = assessment.loc[assessment['Level 4 Government'] == villages.loc[i, 'Level 4 Government'], 'Level 2 Government'].iloc[0]
    villages.loc[i, 'Level 1 Government'] = assessment.loc[assessment['Level 4 Government'] == villages.loc[i, 'Level 4 Government'], 'Level 1 Government'].iloc[0]
    

In [8]:
villages

,Level 4 Government,Count,Level 3 Government,Level 2 Government,Level 1 Government
0,Angokoun-Kpangbassou,38,Languibonou,Botro,Gbeke
1,Andokro-Kouakou,29,Languibonou,Botro,Gbeke
2,Adjougouansou,18,Djibri Assirikro,Sakassou,Gbeke
3,Hannié,18,DJOUROUTOU,Tabou,San Pedro
4,Bledie,17,Dogbo,San-Pédro,San Pedro
...,...,...,...,...,...
132,Tanou-Broukro (Bassin de Saminikro),1,Brobo,Bouaké,Gbeke
133,Ahouzankro (Bassin de Mamini),1,Brobo,Bouaké,Gbeke
134,Takassou (Bassin de Mamini),1,Brobo,Bouaké,Gbeke
135,Koubékro (Bassin Mamini),1,Brobo,Bouaké,Gbeke


In [9]:
# Make a treemap in plotly express to show the distribution of assessments by level 1 government, level 2 government, level 3 government, and level 4 government
fig = px.treemap(villages, path=[px.Constant("Cote d'Ivoire"), 'Level 1 Government', 'Level 2 Government', 'Level 3 Government', 'Level 4 Government'], values='Count')
# change colors to be FF8200 and 009A44
# fig.update_traces(marker=dict(colors=['#FF8200', '#009A44']))
fig.update_layout(
    treemapcolorway = ["#FF8200", "#009A44"],
    # margin = dict(t=50, l=25, r=25, b=25)
)
fig.show()
# export to html
fig.write_html("./home/charts/sites_villages.html")
# write to png
fig.write_image("./home/charts/sites_villages.png")


In [10]:
# write villages to csv
villages.to_csv('./home/charts/sites_villages.csv', encoding='iso-8859-1')

In [11]:
def make_treemap(df, chartfilename, sheetfilename):

    tempdf = pd.DataFrame(df['Level 4 Government'].value_counts())
    tempdf = tempdf.reset_index()
    tempdf.columns = ['Level 4 Government', 'Count']
    # for each value in villages level 4 government, find the corresponding value in the assessment level 4 government column and add the corresponding value in the "level 3 government" column as a new column in the villages dataframe
    for i in range(len(tempdf)):
        tempdf.loc[i, 'Level 3 Government'] = df.loc[df['Level 4 Government'] == tempdf.loc[i, 'Level 4 Government'], 'Level 3 Government'].iloc[0]
        tempdf.loc[i, 'Level 2 Government'] = df.loc[df['Level 4 Government'] == tempdf.loc[i, 'Level 4 Government'], 'Level 2 Government'].iloc[0]
        tempdf.loc[i, 'Level 1 Government'] = df.loc[df['Level 4 Government'] == tempdf.loc[i, 'Level 4 Government'], 'Level 1 Government'].iloc[0]

    # Make a treemap in plotly express to show the distribution of assessments by level 1 government, level 2 government, level 3 government, and level 4 government
    fig = px.treemap(tempdf, path=[px.Constant("Cote d'Ivoire"), 'Level 1 Government', 'Level 2 Government', 'Level 3 Government', 'Level 4 Government'], values='Count')
    # change colors to be FF8200 and 009A44
    # fig.update_traces(marker=dict(colors=['#FF8200', '#009A44']))
    fig.update_layout(
        treemapcolorway = ["#FF8200", "#009A44"]
        )
    fig.show()
    # export to html
    fig.write_html(chartfilename)
    tempdf.to_csv(sheetfilename, encoding='iso-8859-1')
    fig.write_image(chartfilename[:-5]+".png")

In [12]:
# make a new dataframe called healthb that is only rows where the Health access blocked by river is not blank
educationb = assessment[assessment['Health access blocked by river'] != '']
make_treemap(educationb, './home/charts/blocking_health.html', './home/charts/blocking_health.csv')

Index(['Opportunity Unique Identifier', 'Bridge Name', 'Country',
       'Level 1 Government', 'Level 2 Government', 'Level 3 Government',
       'Level 4 Government', 'latitude', 'longitude', 'Reason site not valid',
       'Site Validation Comments', 'Flag for Rejection',
       'River crossing deaths in last 3 years', 'Incident descriptions',
       'Days per year river is flooded', 'Flood duration during rainy season',
       'Notes on social information', 'Width of River During Flooding (m)',
       'Health access blocked by river', 'Education access blocked by river',
       'Market access blocked by river', 'Other access blocked by river',
       'Form: Created Date', 'Form: Created By', 'geometry'],
      dtype='object')

In [13]:
marketb = assessment[assessment['Education access blocked by river'] != '']
make_treemap(marketb, './home/charts/blocking_education.html', './home/charts/blocking_education.csv')

In [14]:
otherb = assessment[assessment['Market access blocked by river'] != '']
make_treemap(otherb, './home/charts/blocking_market.html', './home/charts/blocking_market.csv')

In [15]:
blockingany = assessment[assessment['Other access blocked by river'] != '']
make_treemap(blockingany, './home/charts/blocking_other.html', './home/charts/blocking_other.csv')

In [16]:
blockingany = assessment[(assessment['Health access blocked by river'] != '') | (assessment['Education access blocked by river'] != '') | (assessment['Market access blocked by river'] != '') | (assessment['Other access blocked by river'] != '')]
make_treemap(blockingany, './home/charts/blocking_any.html', './home/charts/blocking_any.csv')

In [17]:
# create a bar chart with villages dataframe where each bar is a government level 3 and the height is the count and color is level 2 government
fig = px.bar(villages, x='Level 3 Government', y='Count', color='Level 2 Government')
# change x title to region of assessment
fig.update_xaxes(title_text='Region of Assessment')
fig.update_xaxes(title_text='Sites Assessed')
fig.update_layout(legend_title_text='Region Legend')
# update title to be Sites Assessed by Region
fig.update_layout(title_text='Sites Assessed by Region')
fig.show()
# write to html
fig.write_html("./home/charts/sites_villages_bar.html")
# write to png
fig.write_image("./home/charts/sites_villages_bar.png")

In [18]:
# create a bar chart with villages dataframe where each bar is a government level 3 and the height is the count and color is level 2 government
tempdf = pd.DataFrame(blockingany['Level 4 Government'].value_counts())
tempdf = tempdf.reset_index()
tempdf.columns = ['Level 4 Government', 'Count']
# for each value in villages level 4 government, find the corresponding value in the assessment level 4 government column and add the corresponding value in the "level 3 government" column as a new column in the villages dataframe
for i in range(len(tempdf)):
    tempdf.loc[i, 'Level 3 Government'] = blockingany.loc[blockingany['Level 4 Government'] == tempdf.loc[i, 'Level 4 Government'], 'Level 3 Government'].iloc[0]
    tempdf.loc[i, 'Level 2 Government'] = blockingany.loc[blockingany['Level 4 Government'] == tempdf.loc[i, 'Level 4 Government'], 'Level 2 Government'].iloc[0]
    tempdf.loc[i, 'Level 1 Government'] = blockingany.loc[blockingany['Level 4 Government'] == tempdf.loc[i, 'Level 4 Government'], 'Level 1 Government'].iloc[0]
fig = px.bar(tempdf, x='Level 3 Government', y='Count', color='Level 2 Government')
# change x title to region of assessment
fig.update_xaxes(title_text='Sites Blocking Healthcare')
fig.update_layout(legend_title_text='Region Legend')
# update title to be Sites Assessed by Region
fig.update_layout(title_text='Sites Blocking Healthcare by Region')
fig.show()
# write to html
fig.write_html("./home/charts/blocking_health_bar.html")
# write to png
fig.write_image("./home/charts/blocking_health_bar.png")

In [19]:
# create a bar chart with villages dataframe where each bar is a government level 3 and the height is the count and color is level 2 government
tempdf = pd.DataFrame(blockingany['Level 4 Government'].value_counts())
tempdf = tempdf.reset_index()
tempdf.columns = ['Level 4 Government', 'Count']
# for each value in villages level 4 government, find the corresponding value in the assessment level 4 government column and add the corresponding value in the "level 3 government" column as a new column in the villages dataframe
for i in range(len(tempdf)):
    tempdf.loc[i, 'Level 3 Government'] = blockingany.loc[blockingany['Level 4 Government'] == tempdf.loc[i, 'Level 4 Government'], 'Level 3 Government'].iloc[0]
    tempdf.loc[i, 'Level 2 Government'] = blockingany.loc[blockingany['Level 4 Government'] == tempdf.loc[i, 'Level 4 Government'], 'Level 2 Government'].iloc[0]
    tempdf.loc[i, 'Level 1 Government'] = blockingany.loc[blockingany['Level 4 Government'] == tempdf.loc[i, 'Level 4 Government'], 'Level 1 Government'].iloc[0]
# create a bar chart with villages dataframe where each bar is a government level 3 and the height is the count and color is level 2 government
fig = px.bar(tempdf, x='Level 3 Government', y='Count', color='Level 2 Government')
# change x title to region of assessment
fig.update_xaxes(title_text='Sites Assessed')
fig.update_layout(legend_title_text='Region Legend')
# update title to be Sites Assessed by Region
fig.update_layout(title_text='Sites Blocking Education by Region')
fig.show()
# write to html
fig.write_html("./home/charts/blocking_education_bar.html")
# write to png
fig.write_image("./home/charts/blocking_education_bar.png")

In [20]:
# create a bar chart with villages dataframe where each bar is a government level 3 and the height is the count and color is level 2 government
tempdf = pd.DataFrame(blockingany['Level 4 Government'].value_counts())
tempdf = tempdf.reset_index()
tempdf.columns = ['Level 4 Government', 'Count']
# for each value in villages level 4 government, find the corresponding value in the assessment level 4 government column and add the corresponding value in the "level 3 government" column as a new column in the villages dataframe
for i in range(len(tempdf)):
    tempdf.loc[i, 'Level 3 Government'] = blockingany.loc[blockingany['Level 4 Government'] == tempdf.loc[i, 'Level 4 Government'], 'Level 3 Government'].iloc[0]
    tempdf.loc[i, 'Level 2 Government'] = blockingany.loc[blockingany['Level 4 Government'] == tempdf.loc[i, 'Level 4 Government'], 'Level 2 Government'].iloc[0]
    tempdf.loc[i, 'Level 1 Government'] = blockingany.loc[blockingany['Level 4 Government'] == tempdf.loc[i, 'Level 4 Government'], 'Level 1 Government'].iloc[0]
fig = px.bar(tempdf, x='Level 3 Government', y='Count', color='Level 2 Government')
# change x title to region of assessment
fig.update_xaxes(title_text='Sites Assessed')
fig.update_layout(legend_title_text='Region Legend')
# update title to be Sites Assessed by Region
fig.update_layout(title_text='Sites Blocking Markets by Region')
fig.show()
# write to html
fig.write_html("./home/charts/blocking_market_bar.html")
# write to png
fig.write_image("./home/charts/blocking_market_bar.png")

In [23]:
# create a bar chart with villages dataframe where each bar is a government level 3 and the height is the count and color is level 2 government
tempdf = pd.DataFrame(blockingany['Level 4 Government'].value_counts())
tempdf = tempdf.reset_index()
tempdf.columns = ['Level 4 Government', 'Count']
# for each value in villages level 4 government, find the corresponding value in the assessment level 4 government column and add the corresponding value in the "level 3 government" column as a new column in the villages dataframe
for i in range(len(tempdf)):
    tempdf.loc[i, 'Level 3 Government'] = blockingany.loc[blockingany['Level 4 Government'] == tempdf.loc[i, 'Level 4 Government'], 'Level 3 Government'].iloc[0]
    tempdf.loc[i, 'Level 2 Government'] = blockingany.loc[blockingany['Level 4 Government'] == tempdf.loc[i, 'Level 4 Government'], 'Level 2 Government'].iloc[0]
    tempdf.loc[i, 'Level 1 Government'] = blockingany.loc[blockingany['Level 4 Government'] == tempdf.loc[i, 'Level 4 Government'], 'Level 1 Government'].iloc[0]
fig = px.bar(tempdf, x='Level 3 Government', y='Count', color='Level 2 Government')
# change x title to region of assessment
fig.update_xaxes(title_text='Sites Assessed')
fig.update_layout(legend_title_text='Region Legend')
# update title to be Sites Assessed by Region
fig.update_layout(title_text='Sites Blocking Other Infrastructure by Region')
fig.show()
# write to html
fig.write_html("./home/charts/blocking_other_bar.html")
# write to png
fig.write_image("./home/charts/blocking_other_bar.png")

In [22]:
# create a bar chart with villages dataframe where each bar is a government level 3 and the height is the count and color is level 2 government
tempdf = pd.DataFrame(blockingany['Level 4 Government'].value_counts())
tempdf = tempdf.reset_index()
tempdf.columns = ['Level 4 Government', 'Count']
# for each value in villages level 4 government, find the corresponding value in the assessment level 4 government column and add the corresponding value in the "level 3 government" column as a new column in the villages dataframe
for i in range(len(tempdf)):
    tempdf.loc[i, 'Level 3 Government'] = blockingany.loc[blockingany['Level 4 Government'] == tempdf.loc[i, 'Level 4 Government'], 'Level 3 Government'].iloc[0]
    tempdf.loc[i, 'Level 2 Government'] = blockingany.loc[blockingany['Level 4 Government'] == tempdf.loc[i, 'Level 4 Government'], 'Level 2 Government'].iloc[0]
    tempdf.loc[i, 'Level 1 Government'] = blockingany.loc[blockingany['Level 4 Government'] == tempdf.loc[i, 'Level 4 Government'], 'Level 1 Government'].iloc[0]
fig = px.bar(tempdf, x='Level 3 Government', y='Count', color='Level 2 Government')
# change x title to region of assessment
fig.update_xaxes(title_text='Sites Assessed')
fig.update_layout(legend_title_text='Region Legend')
# update title to be Sites Assessed by Region
fig.update_layout(title_text='Sites Blocking Any Infrastructure by Region')
fig.show()
# write to html
fig.write_html("./home/charts/blocking_any_bar.html")
# write to png
fig.write_image("./home/charts/blocking_any_bar.png")